**Library Imports**

In [103]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

In [104]:
import os
import sys
print(sys.version)   # Python version information

3.4.4 |Anaconda 2.4.0 (64-bit)| (default, Jan 11 2016, 13:54:01) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


In [105]:
from sympy import *  # Symbolic mathematics
import sympy

In [106]:
#import numba        # Decorators for speeding up python functions

In [107]:
#from astropy import constants  # Physical Constants
#from astropy import units      # Physical Units

In [108]:
import matplotlib as mpl  # Standard plotting library
import matplotlib.pyplot as plt 
import seaborn as sns     # Statistical plotting and improved plot style
#import bokeh             # Interactive visualization library

In [109]:
from sympy.utilities.autowrap import ufuncify  # Creates numerical methods 
                                               # from symbolic expressions

In [110]:
from sympy import abc       # Symbolic variables
from sympy import pi as Pi  # Symbolic pi
from sympy import I as I    # Symbolic imaginary unit

**Display**

In [111]:
from IPython.display import display

In [112]:
from IPython.display import set_matplotlib_formats  
set_matplotlib_formats('pdf', 'png')

In [113]:
import ipywidgets

**Settings & Options**

In [114]:
import numpy as np   # N-Dimensional arrays and numerical routines
import scipy         # Numerical routines

np.set_printoptions(precision=5, threshold=100, edgeitems=3, linewidth=75, 
                    suppress=False, nanstr=None, infstr=None, 
                    formatter=None)

In [115]:
import pandas as pd  # Data structures and data analysis tools

pandas_options = {'display.chop_threshold': None,
                  'display.precision': 4,
                  'display.max_columns': 8,
                  'display.max_rows': 8,
                  'display.max_info_columns': 8,
                  'display.max_info_rows': 8}
for key, val in pandas_options.items():
    pd.set_option(key, val)

In [116]:
sympy.interactive.printing.init_printing(use_latex='mathjax')

In [117]:
# os.system('ipython nbconvert --to pdf {filename}.ipynb')

**Custom functions**

In [118]:
latex_path = os.path.join(os.getcwd(), 'latex', 'figures')
def figure_path(fname): 
    return os.path.join(latex_path, fname)
print(os.getcwd())

/home/jaan/Dropbox/Projects/Crowd-Dynamics/documentation/notebooks


# Field

## Walls

### Round wall

### Linear wall

In [119]:
q = symbols(['x_0:2', 'y_0:2'])
theta = symbols('theta')
rot = Matrix([[cos(theta), -sin(theta)],
              [sin(theta), cos(theta)]])
rot90 = rot.subs(theta, Pi/2)
arg = symbols('t_0:2')
t = Matrix(arg)
A = Matrix.hstack(rot90 * t, -t)
A_inv = simplify(A **-1)
A_inv = A_inv.replace(t[0]**2 + t[1]**2, 1)

In [120]:
q = Matrix(q)

In [121]:
A_inv * q

⎡t₀⋅y₀ - t₁⋅x₀   t₀⋅y₁ - t₁⋅x₁ ⎤
⎢                              ⎥
⎣-t₀⋅x₀ - t₁⋅y₀  -t₀⋅x₁ - t₁⋅y₁⎦

#### Construction
Construct linear wall from start and end points. 
\begin{align}
(\mathbf{p}_{0}, \mathbf{p}_{1}) \mapsto (\mathbf{p}_{0}, \mathbf{p}_{1}, \hat{\mathbf{t}}_{w}, \hat{\mathbf{n}}_{w}, l_w)
\end{align}

In [122]:
linear_params = [
     [[0, 0], [0, 4]],
     [[0, 0], [4, 0]],
     [[0, 4], [4, 4]]
]

In [123]:
def construct_linear_wall(linear_params):
    """

    :param linear_params: Iterable of start and end points.
    :return: Linear wall parameters in array.
    """
    # 90 degree counterclockwise rotation
    rot90 = np.array([[0, -1], [1,  0]])

    rows = len(linear_params)
    cols = 9

    wall = np.zeros((rows, cols))
    for i, p in enumerate(linear_params):
        p = np.array(p)
        d = p[1] - p[0]
        l_w = np.hypot(*d)
#         l_w = np.sqrt(np.dot(d, d))  # Length of the wall
        if l_w == 0:
            raise ValueError(
                "P_0 = {} must not be equal to P-1 = {}".format(p[0], p[1]))
        t_w = d / l_w                # Tangential (unit)vector
        n_w = np.dot(rot90, t_w)     # Normal (unit)vector
        l_w = np.array([l_w])
        wall[i, :] = np.hstack([p[0], p[1], t_w, n_w, l_w])
    return wall

In [124]:
construct_linear_wall([])

array([], shape=(0, 9), dtype=float64)

In [125]:
lw = construct_linear_wall(linear_params)

In [126]:
lw

array([[ 0.,  0.,  0.,  4.,  0.,  1., -1.,  0.,  4.],
       [ 0.,  0.,  4.,  0.,  1.,  0.,  0.,  1.,  4.],
       [ 0.,  4.,  4.,  4.,  1.,  0.,  0.,  1.,  4.]])

deconstruct

In [127]:
w = lw[0]
p_0 = w[0:2]
p_1 = w[2:4]
t_w = w[4:6]
n_w = w[6:8]
l_w = w[8]

In [128]:
p_0, p_1, t_w, n_w, l_w

(array([ 0.,  0.]),
 array([ 0.,  4.]),
 array([ 0.,  1.]),
 array([-1.,  0.]),
 4.0)

In [130]:
import numba

In [137]:
@numba.jit(nopython=True, nogil=True)
def f_iw_linear_tot(linear_wall):
    force = np.zeros(2)
    # TODO: wall object unpacking
    for i in range(len(linear_wall)):
        w = linear_wall[i]
        p_0 = w[0:2]
        p_1 = w[2:4]
        t_w = w[4:6]
        n_w = w[6:8]
        l_w = w[8]
    return force

In [138]:
f_iw_linear_tot(lw)

array([ 0.,  0.])

In [141]:
np.info(lw)

class:  ndarray
shape:  (3, 9)
strides:  (72, 8)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x2dc1350
byteorder:  little
byteswap:  False
type: float64


In [144]:
arr = np.array([1, 1])
arr2 = np.array([[1, 2],
                 [3, 4]])
np.dot(arr, arr2)

array([4, 6])

In [146]:
np.dot(arr, arr2[:, 0]), np.dot(arr, arr2[:, 1])

(4, 6)

In [150]:
np.hypot(arr[0], arr[1])

1.41421356237

## Agents

### Set Positions